Copyright (c) Microsoft Corporation. All rights reserved.

Licensed under the MIT License.

# Lab 3: Automated Machine Learning with AutomML
_**Classification with Local Compute**_

## Introduction
In this lab, you will use your AdeventureWorks model training feature data you created earlier, to let AutoML find the best performing model.

In this notebook you will learn how to:
1. Create an `Experiment` in an existing `Workspace`.
2. Configure AutoML using `AutoMLConfig`.
3. Train the model using local compute.
4. Explore the results.
5. Test the best fitted model.

## Setup

As part of the setup you have already created an Azure ML `Workspace` object. For AutoML you will need to create an `Experiment` object, which is a named object in a `Workspace` used to run experiments.

In [ ]:
import azureml.core
print(azureml.core.VERSION)

## Connect to your Azure Machine Learning service workspace.

There are two ways to do this:

- Using the workspace config file we created earlier.
- Calling the workspace get() method 

In [ ]:
# Method 1: Using the workspace config file...

ws = Workspace.from_config()
print(ws.name, ws.location, ws.resource_group, ws.location, sep='\t')

In [ ]:
# Method 2: Using the get() method...

from azureml.core import Workspace, Experiment, Run
ws = Workspace.get(name='dp100labws',
                   subscription_id='<your-subscription_id>',
                   resource_group='dp100labrg'
                  )
print(ws.name, ws.resource_group, ws.location, sep='\t')

In [ ]:
import logging

from matplotlib import pyplot as plt
import numpy as np
import pandas as pd

import azureml.core
from azureml.core.experiment import Experiment
from azureml.core.workspace import Workspace
from azureml.train.automl import AutoMLConfig
print('Modules loaded...')

In [ ]:
#ws = Workspace.from_config()

# Choose a name for the experiment and specify the project folder.
experiment_name = 'dp100lab-automl'
project_folder = './dp100lab'

experiment = Experiment(ws, experiment_name)

output = {}
output['SDK version'] = azureml.core.VERSION
output['Subscription ID'] = ws.subscription_id
output['Workspace Name'] = ws.name
output['Resource Group'] = ws.resource_group
output['Location'] = ws.location
output['Project Directory'] = project_folder
output['Experiment Name'] = experiment.name
pd.set_option('display.max_colwidth', -1)
outputDf = pd.DataFrame(data = output, index = [''])
outputDf.T

## Connect to existing workspace.

# Explore data
### You already explored the data in the last lab. You need to copy the data into the cloud so it can be accessed by your cloud training environment. We saved the model training data to a csv file so all we have to do is load it.

In [ ]:
import pandas as pd
import os

cwd = os.getcwd()

os.chdir("..")
dd = os.path.join(os.getcwd(), 'Data')
os.chdir(cwd)

df_features = pd.read_csv(os.path.join(dd, 'BikeModelFeatures.csv'))
df_features.head()

In [ ]:
from sklearn.model_selection import train_test_split

x_train , x_test = train_test_split(df_features.values,test_size=0.2)       #test_size=0.5(whole_data)

In [ ]:
y_train = x_train[:,3]
y_train

In [ ]:
y_test = x_test[:,3]
y_test

In [ ]:
x_train = x_train[:,0:3]
x_train

In [ ]:
x_test = x_test[:,0:3]
x_test

In [ ]:
print(len(x_train))
len(x_test)

## Train

Instantiate an `AutoMLConfig` object to specify the settings and data used to run the experiment.

|Property|Description|
|-|-|
|**task**|classification or regression|
|**primary_metric**|This is the metric that you want to optimize. Regression supports the following primary metrics: <br><i>spearman_correlation</i><br><i>normalized_root_mean_squared_error</i><br><i>r2_score</i><br><i>normalized_mean_absolute_error</i>|
|**iteration_timeout_minutes**|Time limit in minutes for each iteration.|
|**iterations**|Number of iterations. In each iteration AutoML trains a specific pipeline with the data.|
|**n_cross_validations**|Number of cross validation splits.|
|**X**|(sparse) array-like, shape = [n_samples, n_features]|
|**y**|(sparse) array-like, shape = [n_samples, ], targets values.|
|**path**|Relative path to the project folder. AutoML stores configuration files for the experiment under this folder. You can specify a new empty folder.|

In [ ]:
automl_config = AutoMLConfig(task = 'classification',
                             iteration_timeout_minutes = 10,
                             iterations = 10,
                             primary_metric = 'precision_score_weighted',
                             n_cross_validations = 5,
                             debug_log = 'automl.log',
                             verbosity = logging.INFO,
                             X = x_train, 
                             y = y_train,
                             preprocess=True,
                             path = project_folder)

In [ ]:
print('Done')

Call the `submit` method on the experiment object and pass the run configuration. Execution of local runs is synchronous. Depending on the data and the number of iterations this can run for a while.
In this example, we specify `show_output = True` to print currently running iterations to the console.

In [ ]:
local_run = experiment.submit(automl_config, show_output = True)

In [ ]:
local_run

## Results

#### Widget for Monitoring Runs

The widget will first report a "loading" status while running the first iteration. After completing the first iteration, an auto-updating graph and table will be shown. The widget will refresh once per minute, so you should see the graph update as child runs complete.

**Note:** The widget displays a link at the bottom. Use this link to open a web interface to explore the individual run details.

In [ ]:
from azureml.widgets import RunDetails
RunDetails(local_run).show() 


#### Retrieve All Child Runs
You can also use SDK methods to fetch all the child runs and see individual metrics that we log.

In [ ]:
children = list(local_run.get_children())
metricslist = {}
for run in children:
    properties = run.get_properties()
    metrics = {k: v for k, v in run.get_metrics().items() if isinstance(v, float)}
    metricslist[int(properties['iteration'])] = metrics

rundata = pd.DataFrame(metricslist).sort_index(1)
rundata

### Retrieve the Best Model

Below we select the best pipeline from our iterations. The `get_output` method returns the best run and the fitted model. The Model includes the pipeline and any pre-processing.  Overloads on `get_output` allow you to retrieve the best run and fitted model for *any* logged metric or for a particular *iteration*.

In [ ]:
best_run, fitted_model = local_run.get_output()
print(best_run)
print(fitted_model)

#### Best Model Based on Any Other Metric
Show the run and the model that has the smallest `root_mean_squared_error` value (which turned out to be the same as the one with largest `spearman_correlation` value):

In [ ]:
lookup_metric = "root_mean_squared_error"
best_run, fitted_model = local_run.get_output(metric = lookup_metric)
print(best_run)
print(fitted_model)

#### Model from a Specific Iteration
Show the run and the model from the third iteration:

In [ ]:
iteration = 3
third_run, third_model = local_run.get_output(iteration = iteration)
print(third_run)
print(third_model)

## Test

Predict on training and test set, and calculate model accuracy.

In [ ]:
# calculate accuracy on the prediction
acc = np.average(y_predtest == y_test)
print('Accuracy is', acc)